In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import tensorflow_datasets as tfds

In [0]:
tf.random.set_seed(1228)
np.random.seed(1228)

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=80000)

17465344/17464789 [==============================] - 1s 0us/step


In [5]:
word_index = tf.keras.datasets.imdb.get_word_index()

1646592/1641221 [==============================] - 0s 0us/step


In [6]:
len(word_index)

88584

In [0]:
def length_check(x):
  return max([len(i) for i in x])

In [8]:
length_check(x_train)

2494

In [9]:
maxlength = length_check(x_train)

maxlength

2494

In [0]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, 1000, padding='post')
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, 1000, padding='post')

In [0]:
class layer_parameters:
  kernel_initializer = tf.keras.initializers.GlorotNormal(seed=1228)


layer_param = layer_parameters()

In [0]:
class My_Attention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super(My_Attention, self).__init__(**kwargs)
    self.units = units

    self.w = tf.keras.layers.Dense(units, kernel_initializer=layer_param.kernel_initializer)
    self.u_w = tf.keras.layers.Dense(1, kernel_initializer=layer_param.kernel_initializer, use_bias=False)
    

  def call(self, inputs):
    uit = self.w(inputs)
    uit = tf.nn.tanh(uit)
    ait = tf.nn.softmax(self.u_w(uit), axis=1)
    si = tf.reduce_sum(ait * inputs, axis=1)
    return si, ait


In [0]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if tf.keras.backend.backend() == 'tensorflow':
        return tf.keras.backend.squeeze(tf.keras.backend.dot(x, tf.keras.backend.expand_dims(kernel)), axis=-1)
    else:
        return tf.keras.backend.dot(x, kernel)

class AttentionWithContext(tf.keras.layers.Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = layer_param.kernel_initializer

        self.W_regularizer = tf.keras.regularizers.get(W_regularizer)
        self.u_regularizer = tf.keras.regularizers.get(u_regularizer)
        self.b_regularizer = tf.keras.regularizers.get(b_regularizer)

        self.W_constraint = tf.keras.constraints.get(W_constraint)
        self.u_constraint = tf.keras.constraints.get(u_constraint)
        self.b_constraint = tf.keras.constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='W',
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zero',
                                     name='b',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='u',
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = tf.keras.backend.tanh(uit)
        ait = dot_product(uit, self.u)

        a = tf.keras.backend.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= tf.keras.backend.cast(mask, tf.keras.backend.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= tf.keras.backend.cast(tf.keras.backend.sum(a, axis=1, keepdims=True) + tf.keras.backend.epsilon(), tf.keras.backend.floatx())

        a = tf.keras.backend.expand_dims(a)
        weighted_input = x * a
        return tf.keras.backend.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
len(word_index)

88584

In [0]:
text_input = tf.keras.layers.Input(shape=(1000,))
emb = tf.keras.layers.Embedding(80000+1, 256)(text_input)
bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(emb)

my_attention = My_Attention(512)

matt, weight = my_attention(bi_lstm)
or_attention = AttentionWithContext()(bi_lstm)
plane = tf.keras.layers.Flatten()(bi_lstm)

ma_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Brief_Att')(matt)
or_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Full_Att')(or_attention)
pl_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Plane_No_Att')(plane)


model = tf.keras.models.Model(text_input, [ma_out, or_out, pl_out])

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1000, 256)    20480256    input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 1000, 512)    1050624     embedding[0][0]                  
__________________________________________________________________________________________________
my__attention (My_Attention)    ((None, 512), (None, 263168      bidirectional[0][0]              
______________________________________________________________________________________________

In [0]:
model.compile(loss={'Brief_Att':'binary_crossentropy', 'Full_Att':'binary_crossentropy', 'Plane_No_Att':'binary_crossentropy'},
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics={'Brief_Att':'acc', 'Full_Att':'acc', 'Plane_No_Att':'acc'})

In [0]:
hist = model.fit(x_train, [y_train, y_train, y_train], batch_size=32, epochs=100, validation_data=(x_test, [y_test, y_test, y_test]))

Train on 25000 samples, validate on 25000 samples
Epoch 1/100
25000/25000 [==============================] - 429s 17ms/sample - loss: 1.4972 - Brief_Att_loss: 0.5112 - Full_Att_loss: 0.5163 - Plane_No_Att_loss: 0.4700 - Brief_Att_acc: 0.7579 - Full_Att_acc: 0.7490 - Plane_No_Att_acc: 0.7718 - val_loss: 1.3713 - val_Brief_Att_loss: 0.3182 - val_Full_Att_loss: 0.3221 - val_Plane_No_Att_loss: 0.7305 - val_Brief_Att_acc: 0.8764 - val_Full_Att_acc: 0.8759 - val_Plane_No_Att_acc: 0.7584
Epoch 2/100
25000/25000 [==============================] - 418s 17ms/sample - loss: 0.6971 - Brief_Att_loss: 0.2323 - Full_Att_loss: 0.2336 - Plane_No_Att_loss: 0.2307 - Brief_Att_acc: 0.9184 - Full_Att_acc: 0.9186 - Plane_No_Att_acc: 0.9146 - val_loss: 0.8999 - val_Brief_Att_loss: 0.2955 - val_Full_Att_loss: 0.2964 - val_Plane_No_Att_loss: 0.3085 - val_Brief_Att_acc: 0.8772 - val_Full_Att_acc: 0.8774 - val_Plane_No_Att_acc: 0.8715
Epoch 3/100
25000/25000 [==============================] - 417s 17ms/sample - 

# Loss  - Mask

In [0]:
text_input = tf.keras.layers.Input(shape=(1000,))
emb = tf.keras.layers.Embedding(80000+1, 256, mask_zero=True)(text_input)
bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(emb)

my_attention = My_Attention(512)

matt, weight = my_attention(bi_lstm)
or_attention = AttentionWithContext()(bi_lstm)
plane = tf.keras.layers.Flatten()(bi_lstm)

ma_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Brief_Att')(matt)
or_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Full_Att')(or_attention)
pl_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Plane_No_Att')(plane)


model = tf.keras.models.Model(text_input, [ma_out, or_out, pl_out])

In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 256)    20480256    input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1000, 512)    1050624     embedding_1[0][0]                
__________________________________________________________________________________________________
my__attention_1 (My_Attention)  ((None, 512), (None, 263168      bidirectional_1[0][0]            
____________________________________________________________________________________________

In [0]:
model.compile(loss={'Brief_Att':'binary_crossentropy', 'Full_Att':'binary_crossentropy', 'Plane_No_Att':'binary_crossentropy'},
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics={'Brief_Att':'acc', 'Full_Att':'acc', 'Plane_No_Att':'acc'})

In [0]:
hist = model.fit(x_train, [y_train, y_train, y_train], batch_size=32, epochs=100, validation_data=(x_test, [y_test, y_test, y_test]))

Train on 25000 samples, validate on 25000 samples
Epoch 1/100
25000/25000 [==============================] - 365s 15ms/sample - loss: 1.4657 - Brief_Att_loss: 0.5713 - Full_Att_loss: 0.4597 - Plane_No_Att_loss: 0.4363 - Brief_Att_acc: 0.8014 - Full_Att_acc: 0.8153 - Plane_No_Att_acc: 0.8132 - val_loss: 0.9474 - val_Brief_Att_loss: 0.3204 - val_Full_Att_loss: 0.3030 - val_Plane_No_Att_loss: 0.3239 - val_Brief_Att_acc: 0.8766 - val_Full_Att_acc: 0.8781 - val_Plane_No_Att_acc: 0.8784
Epoch 2/100
25000/25000 [==============================] - 352s 14ms/sample - loss: 0.7865 - Brief_Att_loss: 0.3080 - Full_Att_loss: 0.2332 - Plane_No_Att_loss: 0.2447 - Brief_Att_acc: 0.9108 - Full_Att_acc: 0.9162 - Plane_No_Att_acc: 0.9170 - val_loss: 1.2421 - val_Brief_Att_loss: 0.4780 - val_Full_Att_loss: 0.3336 - val_Plane_No_Att_loss: 0.4313 - val_Brief_Att_acc: 0.8732 - val_Full_Att_acc: 0.8760 - val_Plane_No_Att_acc: 0.8732
Epoch 3/100
25000/25000 [==============================] - 352s 14ms/sample - 

## With Drop out

In [0]:
text_input = tf.keras.layers.Input(shape=(1000,))
emb = tf.keras.layers.Embedding(80000+1, 256, mask_zero=True)(text_input)
emb = tf.keras.layers.SpatialDropout1D(0.3)(emb)
bi_lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, recurrent_dropout=0.3, dropout=0.3))(emb)

my_attention = My_Attention(512)

matt, weight = my_attention(bi_lstm)
or_attention = AttentionWithContext()(bi_lstm)
plane = tf.keras.layers.Flatten()(bi_lstm)

ma_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Brief_Att')(matt)
or_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Full_Att')(or_attention)
pl_out = tf.keras.layers.Dense(1, activation='sigmoid', name='Plane_No_Att')(plane)


model = tf.keras.models.Model(text_input, [ma_out, or_out, pl_out])

In [0]:
model.compile(loss={'Brief_Att':'binary_crossentropy', 'Full_Att':'binary_crossentropy', 'Plane_No_Att':'binary_crossentropy'},
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics={'Brief_Att':'acc', 'Full_Att':'acc', 'Plane_No_Att':'acc'})

In [0]:
hist = model.fit(x_train, [y_train, y_train, y_train], batch_size=32, epochs=100, validation_data=(x_test, [y_test, y_test, y_test]))

Train on 25000 samples, validate on 25000 samples
Epoch 1/100
25000/25000 [==============================] - 4461s 178ms/sample - loss: 1.6694 - Brief_Att_loss: 0.6302 - Full_Att_loss: 0.5473 - Plane_No_Att_loss: 0.4918 - Brief_Att_acc: 0.7319 - Full_Att_acc: 0.7785 - Plane_No_Att_acc: 0.7756 - val_loss: 1.2943 - val_Brief_Att_loss: 0.5558 - val_Full_Att_loss: 0.4027 - val_Plane_No_Att_loss: 0.3357 - val_Brief_Att_acc: 0.8316 - val_Full_Att_acc: 0.8684 - val_Plane_No_Att_acc: 0.8706
Epoch 2/100
25000/25000 [==============================] - 4364s 175ms/sample - loss: 1.0575 - Brief_Att_loss: 0.4632 - Full_Att_loss: 0.3019 - Plane_No_Att_loss: 0.2917 - Brief_Att_acc: 0.8532 - Full_Att_acc: 0.8961 - Plane_No_Att_acc: 0.8952 - val_loss: 0.8948 - val_Brief_Att_loss: 0.3143 - val_Full_Att_loss: 0.2815 - val_Plane_No_Att_loss: 0.2990 - val_Brief_Att_acc: 0.8716 - val_Full_Att_acc: 0.8847 - val_Plane_No_Att_acc: 0.8812
Epoch 3/100
25000/25000 [==============================] - 4415s 177ms/sam

## **)

In [0]:
# The Plane used a glovbal max pool <- does not identifiy the correct way to compare the result

hist = model.fit(x_train, [y_train, y_train, y_train], batch_size=32, epochs=100, validation_data=(x_test, [y_test, y_test, y_test]))

Train on 25000 samples, validate on 25000 samples
Epoch 1/100
25000/25000 [==============================] - 371s 15ms/sample - loss: 1.4275 - Brief_Att_loss: 0.4897 - Full_Att_loss: 0.4922 - Plane_No_Att_loss: 0.4462 - Brief_Att_acc: 0.7403 - Full_Att_acc: 0.7398 - Plane_No_Att_acc: 0.7806 - val_loss: 0.8721 - val_Brief_Att_loss: 0.2870 - val_Full_Att_loss: 0.2871 - val_Plane_No_Att_loss: 0.2980 - val_Brief_Att_acc: 0.8821 - val_Full_Att_acc: 0.8821 - val_Plane_No_Att_acc: 0.8754
Epoch 2/100
25000/25000 [==============================] - 368s 15ms/sample - loss: 0.5993 - Brief_Att_loss: 0.1978 - Full_Att_loss: 0.1974 - Plane_No_Att_loss: 0.2036 - Brief_Att_acc: 0.9262 - Full_Att_acc: 0.9273 - Plane_No_Att_acc: 0.9233 - val_loss: 0.8566 - val_Brief_Att_loss: 0.2836 - val_Full_Att_loss: 0.2840 - val_Plane_No_Att_loss: 0.2900 - val_Brief_Att_acc: 0.8858 - val_Full_Att_acc: 0.8857 - val_Plane_No_Att_acc: 0.8856
Epoch 3/100
25000/25000 [==============================] - 369s 15ms/sample - 

KeyboardInterrupt: ignored